In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bamt
import typing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, QuantileTransformer
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

In [2]:
"""
Применим биннинг
"""
def binning(x: pd.Series, thresh: int = 5) -> typing.Union[typing.List, pd.Series]:
    if x.name[:-2] == 'duration_treatment':
        return pd.qcut(x, 8, labels=False, duplicates='drop')

    if x.unique().shape[0] > thresh:
        return pd.qcut(x, thresh, labels=False, duplicates='drop')
    else:
        return round(x)


def fill_with_mice(x: pd.Series):
    lr = LinearRegression()
    imp = IterativeImputer(estimator=lr,missing_values=np.nan, 
                           max_iter=50, 
                           verbose=0, 
                           imputation_order='roman',
                           skip_complete=True,
                           add_indicator=True,
                           random_state=0)
    return pd.Series(imp.fit_transform(x.to_numpy().reshape(1, -1))[0])

In [3]:
df = pd.read_csv(r'data\patients_data_7_to_7_with_therapy.csv')

In [4]:
#Создаем словарь, при сохранении матрицы смежности, переводим фичи на английский
translation_dict = {
     'Фибриноген': 'Fibrinogen',
     'Калий': 'Potassium',
     'PCT- Тромбокрит' : 'PCT- Thrombocrit',
     'Альбумин' : 'Albumen',
     'ощущение заложенности в грудной клетке': 'Feeling of congestion in the chest',
     'Гранулоциты#' : 'Granulocytes#',
     'HGB- Гемоглобин': 'HGB',
     'WBC- Лейкоциты': 'WBC',
     'ОАМ Уробилиноген': 'Urinalysis Urobilinogen',
     'P-LCR- Отнош.крупных тр. к общ.кол-ву': 'P-LCR',
     'К+ (вена)': 'K+ (vein)',
     'степень_тяжести_течения': 'Severity of the course',
     'ОАМ Удельная плотность': 'Urinalysis Specific Density',
     'Вес': 'Weight',
     'Эозинофилы#': 'Eosinophils',
     'ОАМ Кислотность (прибор)': 'Urinalysis Acidity',
     'Площадь поверхности тела': 'Body surface area',
     'АСТ': 'AST',
     'Кислородотерапия': 'Oxygen therapy',
     'Состояние_пациента': 'Patient condition',
     'АЧТВ': 'APTT',
     'RBC- Эритроциты': 'RBC',
     'Общий белок': 'Total protein',
     'ЧСС': 'Heart rate',
     'АЛТ': 'Alanineaminotransferase',
     'Нейтрофилы#': 'Neutrophils#',
     'PLT- Тромбоциты': 'PLT',
     'Незрелые гранулоциты (IG%)': 'IG%',
     'С-реактивный белок (СРБ)': 'C-reactive protein',
     'Эозинофилы%': 'Eosinophils%',
     'статус_ковида': 'COVID-19 status',
     'MPV- Средн.объем тромбоцитов': 'MPV',
     'Моноциты%': 'Monocytes%',
     'ОАМ Билирубин': 'Urinalysis Bilirubin',
     'С-реактивный белок': 'C-reactive protein',
     'Пневмония': 'Pneumonia',
     'HCT- Гематокрит': 'HCT',
     'Положительный результат лабораторного исследования на наличие РНК SARS-CoV-2 с применением методов амплификации нуклеиновых кислот вне зависимости от клинических проявлений': 'positive SARS-CoV-2',
     'Cтепень тяжести по КТ': 'Severity according to CT scan' ,
     'ОАМ Белок': 'Urinalysis Protein',
     'ранее_болел_ковидом': 'Had COVID previously',
     'температура тела выше 37,5 °C': 'Body temperature higher than 37,5 °C',
     'С-реактивный белок (СРБ) колич.': 'C-reactive protein #',
     'Температура': 'Temperature',
     'возвращение из зарубежной поездки за 14 дней до появления симптомов': 'Returning from a foreign trip 14 days before the onset of symptoms',
     'Д-димер': 'D-dimer',
     'ЧДД': 'Respiratory rate',
     'Билирубин общий': 'Bilirubin total',
     'Мочевина': 'Urea',
     'Лактатдегидрогеназа': 'Lactatedehydrogenase',
     'ОАМ Лейкоциты': 'Urinalysis leukocytes',
     'dad': 'dad',
     'Наличие клинических проявлений тяжелой пневмонии': 'The presence of clinical manifestations of severe pneumonia',
     'Лимфоциты#': 'Lymphocytes#',
     'Глюкоза': 'Glucose',
     'Процент поражения легочной ткани': 'Percentage of lung tissue damage',
     'Пульс': 'Pulse',
     'Лимфоциты%': 'Lymphocytes%',
     'вакцинировался_от_пневмонии': 'Vaccinated against pneumonia',
     'Базофилы#': 'Basophils',
     'PDW- Индекс расп.по объему тр.': 'PDW',
     'Сатурация': 'Saturation',
     'головная боль': 'Headache',
     'тип_дыхания': 'Type of breathing',
     'Протромбин (по Квику)': 'Prothrombin (according to Quick)',
     'вакцинировался_от_гриппа': 'Vaccinated against influenza',
     'Ферритин': 'Ferritin',
     'Моноциты#': 'Monocytes#',
     'ОАМ Эритроциты': 'Urinalysis erythrocytes',
     'Протромбиновое время': 'Prothrombin time',
     'Креатинин': 'Creatinine',
     'ОАМ Глюкоза': 'Urinalysis glucose',
     'ОАМ Кетоновые тела': 'Urinalysis Ketone bodie',
     'Прокальцитониновый тест': 'Procalcitonin test',
     'Лактат': 'Lactate',
     'MCV- Средн.объем эритроцитов': 'MCV',
     'ИБС': 'Coronary artery disease',
     'cad': 'cad',
     'Рост': 'Height',
     'Гранулоциты%': 'Granulocytes%',
     'снижение_сознания': 'Decrease of consciousness',
     'Нейтрофилы%': 'Neutrophils',
     'Незрелые гранулоциты (IG#)': 'IG#',
     'Базофилы%': 'Basophils%',
     'МНО': 'INR',
     'outcome': 'Uutcome',
     'result': 'Result',
     'duration_treatment': 'Duration_treatment',
     'duration_diseases': 'Duration_diseases',
     'age': 'Age',
     'gender': 'Gender',
     'Лопинавир+Ритонавир': 'Lopinavir and Ritonavir ',
     'Азитромицин': 'Azithromycin ', 
     'Интерферон_бета-1b': 'Interferon_beta_b',
     'Бисопролол': 'Bisoprolol',
     'Бисопролол-Тева': 'Bisoprolol-Teva', 
     'Бисопролол-Прана': 'Bisoprolol-Prana', 
     'Бисопролол-Лугал': 'Bisoprolol-Lugal', 
     'Карведилол': 'Carvedilol',
     'Небиволол': 'Nebivolol', 
     'Метопролол': 'Metoprolol',
     'Гепарин-натрий_Браун': 'Heparin-sodium Brown ',
     'Гепарин': 'Heparin', 
     'Парацетамол': 'Paracetamol'
     
}

In [5]:
#Рассмотрим только пациентов с позитивными и негативными исходами
df = df.drop(index = df[df.result== 0].index).reset_index(drop=True)

df.result[df.result == 1] = 0
df.result[df.result == 2] = 1

C:\Users\21306\AppData\Local\Temp\ipykernel_7912\680662705.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.result[df.result == 1] = 0
C:\Users\21306\AppData\Local\Temp\ipykernel_7912\680662705.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.result[df.result == 2] = 1


In [6]:
#Пробую улучшить duration_treatment
#Выбираю признаки, которые уменьшают ошибку по Duration_treatment

#Признаки были отобраны с помощью RFE(Recursive feature elimination)

df = df[['Unnamed: 0','t_point',
       'Протромбин (по Квику)', 'Незрелые гранулоциты (IG#)',
       'Билирубин общий', 'PLT- Тромбоциты', 'МНО', 'вакцинировался_от_гриппа',
       'RBC- Эритроциты', 'MCV- Средн.объем эритроцитов',
       'PDW- Индекс расп.по объему тр.', 'Нейтрофилы#',
       'С-реактивный белок (СРБ) колич.',
       'ощущение заложенности в грудной клетке', 'Нейтрофилы%',
       'Состояние_пациента', 'Cтепень тяжести по КТ', 'С-реактивный белок',
       'ОАМ Глюкоза', 'Мочевина', 'насморк и другие катаральные симптомы',
       'Креатинин', 'Балл по NEWS', 'P-LCR- Отнош.крупных тр. к общ.кол-ву',
       'Пульс', 'Лимфоциты%', 'Рост', 'АСТ', 'ЧСС', 'Сатурация',
       'Гранулоциты#', 'PCT- Тромбокрит', 'WBC- Лейкоциты', 'Фибриноген',
       'Лактатдегидрогеназа', 'ОАМ Эритроциты', 'ОАМ Уробилиноген',
       'Базофилы#', 'АЛТ', 'Ферритин', 'слабость.1', 'HGB- Гемоглобин',
       'Процент поражения легочной ткани', 'MPV- Средн.объем тромбоцитов',
       'Моноциты%', 'outcome', 'age', 'gender','result',
       'Гидроксихлорохин', 'Фраксипарин', 'Клексан', 'Амброксол',
       'Интерферон_бета-1b', 'Бисопролол', 'Метопролол',
       'duration_treatment']]

In [7]:
#Разделяю на df 4 временных интервалов
df_t0 = df[df.t_point == 't_0'][df[df.t_point == 't_0'].columns[2:]]
df_t1 = df[df.t_point == 't_1'][df[df.t_point == 't_1'].columns[2:]]
df_t2 = df[df.t_point == 't_2'][df[df.t_point == 't_2'].columns[2:]]
df_t3 = df[df.t_point == 't_3'][df[df.t_point == 't_3'].columns[2:]]

In [8]:
#Беру ненормализованные значения duration_treatment, чтобы позже добавить
dur_t0 = df_t0.duration_treatment
dur_t1 = df_t1.duration_treatment
dur_t2 = df_t2.duration_treatment
dur_t3 = df_t3.duration_treatment

In [9]:
#Функция заполняет пропущенные значения, дискритизирует и разделяет на train, test
def create_train_test_dataframes(df: pd.DataFrame,
                                 time_moment: int, test_thresh: int = 0.3) -> typing.Tuple:


    df = df.apply(fill_with_mice)

    df = df.apply(binning)

    df['mock'] = [0 for i in range(df.shape[0])]
    X_train, X_test, y_train, y_test = train_test_split(df,
                                                        df['mock'], 
                                                        test_size=test_thresh)

    X_train.drop(columns=['mock', f'outcome'], inplace=True)

    X_train.dropna(axis=1, inplace=True)
    X_train = X_train.applymap(int)
    X_test.drop(columns=['mock', f'outcome'], inplace=True)


    X_test.dropna(axis=1, inplace=True)
    X_test = X_test.applymap(int)
    return X_train, X_test


In [10]:
#Разделяю фреймы на test, train
train_t0,test_t0 = create_train_test_dataframes(df_t0, 0)
train_t1,test_t1 = create_train_test_dataframes(df_t1, 0)
train_t2,test_t2 = create_train_test_dataframes(df_t2, 0)
train_t3,test_t3 = create_train_test_dataframes(df_t3, 0)

In [11]:
#Добавляю ненормализированный Duration_treatment
def duration_treatment_real(train_t0,df):
    train_t0.duration_treatment = df.duration_treatment.iloc[train_t0.duration_treatment.index].values
    return train_t0

#train
train_t0 = duration_treatment_real(train_t0,df)
train_t1 = duration_treatment_real(train_t1,df)
train_t2 = duration_treatment_real(train_t2,df)
train_t3 = duration_treatment_real(train_t3,df)
# train_t4 = duration_treatment_real(train_t4,df)

#test
test_t0 = duration_treatment_real(test_t0,df)
test_t1 = duration_treatment_real(test_t1,df)
test_t2 = duration_treatment_real(test_t2,df)
test_t3 = duration_treatment_real(test_t3,df)
# test_t4 = duration_treatment_real(test_t4,df)

In [12]:
import bamt.Networks as Nets
import bamt.Preprocessors as p

from sklearn import preprocessing
import matplotlib.pyplot as plt
from pgmpy.estimators import K2Score

#Создаем сеть

encoder = preprocessing.LabelEncoder()
discretizer = preprocessing.KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='quantile')
p = p.Preprocessor([('encoder', encoder), ('discretizer', discretizer)])


def create_bn(df: pd.DataFrame):
    
    discretized_data, est = p.apply(df)
    bn = Nets.DiscreteBN() # init BN
    bn.add_nodes(p.info)

    bn.add_edges(discretized_data, scoring_function=('BIC',)) # use mutual information sf implemented in BAMT
    bn.fit_parameters(discretized_data)
    print('Done!')
    return bn

In [13]:
#Считаем метрики точности для получившейся модели
from sklearn.metrics import mean_absolute_error
predictions = {}

def evaluate_bn(bn, test_df, indicator):
#     discretized_data, est = p.apply(test_df)
    columns = [f'result',f'duration_treatment']

    validY = test_df[columns]
    validX = test_df.drop(columns, axis=1)
    predictions_mi = bn.predict(test=validX, parall_count=2)
    predictions_mi = {key: list(map(int, value)) for key, value in predictions_mi.items()}
    predictions[indicator] = predictions_mi
    for column in columns:
        if column == f'result':
            print(f"{column: <25} Accuracy:  {accuracy_score(validY[column], predictions_mi[column])}")
            print(f"{column: <25} F1 score:  {f1_score(validY[column], predictions_mi[column], average='weighted')}")
        else:
            print(f"{column: <25} MAE:  {mean_absolute_error(validY[column], predictions_mi[column])}")
    return predictions

In [14]:
#Сохраняем матрицу смежности и переводим фичи на английский
def save_adjmat(bn,n):
    bn_data = {}

    with open(f'dbn_adjmat_without_durdis_{n}.csv', 'w') as f:
        for index, bn in enumerate([bn], start=1):
            data = {}
            trans_nodes = []
            i, j = 0, 0
            for edge in bn.edges:
                fst, snd = edge
                try:
                    new_edge = (translation_dict[fst], translation_dict[snd]) #Переводим название фичей
                    f.write(','.join(new_edge) + '\n')
                except:
                    pass
        print('Excellent')

In [15]:
%%time 
bn1 = create_bn(train_t0)
    
save_adjmat(bn1,0)

pred_d0 = evaluate_bn(bn1, test_t0, 1)
df_t0_predict = pd.DataFrame(list(pred_d0.values())[0])
df_t0_predict['patients'] = list(df[df.index.isin(list(test_t0.index))]['Unnamed: 0'])
df_t0_predict.to_csv('df_t0_predict_without_durdis.csv')

2022-11-22 20:20:01,389 | INFO     | Preprocessors.py-scan-0087 | No one column is continuous
Done!
Excellent


100%|██████████| 263/263 [00:11<00:00, 23.23it/s]


result                    Accuracy:  0.7452471482889734
result                    F1 score:  0.8076350128600083
duration_treatment        MAE:  9.543726235741445
CPU times: total: 1min 29s
Wall time: 3min 36s


In [ ]:
%%time 
bn2 = create_bn(train_t1)
    
save_adjmat(bn2,1)

pred_d1 = evaluate_bn(bn2, test_t1, 1)
df_t1_predict = pd.DataFrame(list(pred_d1.values())[0])
df_t1_predict['patients'] = list(df[df.index.isin(list(test_t1.index))]['Unnamed: 0'])
df_t1_predict.to_csv('df_t1_predict_without_durdis.csv')

2022-11-22 20:23:37,608 | INFO     | Preprocessors.py-scan-0087 | No one column is continuous


In [ ]:
%%time 
bn3 = create_bn(train_t2)
    
save_adjmat(bn3,2)

pred_d2 = evaluate_bn(bn3, test_t2, 1)
df_t2_predict = pd.DataFrame(list(pred_d2.values())[0])
df_t2_predict['patients'] = list(df[df.index.isin(list(test_t2.index))]['Unnamed: 0'])
df_t2_predict.to_csv('df_t2_predict_without_durdis.csv')

In [ ]:
%%time 
bn4 = create_bn(train_t3)
    
save_adjmat(bn4,3)

pred_d3 = evaluate_bn(bn4, test_t3, 1)
df_t3_predict = pd.DataFrame(list(pred_d3.values())[0])
df_t3_predict['patients'] = list(df[df.index.isin(list(test_t3.index))]['Unnamed: 0'])
df_t3_predict.to_csv('df_t3_predict_without_durdis.csv')

In [ ]:
#Считаем ментрики точности для разных моделей
def dur_df(test, n):
    #t3
    predictions_mi = pd.read_csv(f'df_t{n}_predict_without_durdis.csv',index_col = [0]).drop(columns = ['patients'])
    columns = [f'result', f'duration_treatment']
    validY = test[columns]
    validX = test.drop(columns, axis=1)
    for column in columns:
        if column == f'result':
            print(f"{column: <25} Accuracy:  {accuracy_score(validY[column], predictions_mi[column])}")
            print(f"{column: <25} F1 score:  {f1_score(validY[column], predictions_mi[column], average='weighted')}")
        else:
            print(f"{column: <25} MAE:  {mean_absolute_error(validY[column], predictions_mi[column])}")
    df_graph = pd.DataFrame()
    df_graph['duration_treatment_true'] = validY['duration_treatment'].values
    df_graph['duration_treatment_pred'] = predictions_mi['duration_treatment'].values
    return pd.DataFrame(df_graph,columns = ['duration_treatment_true','duration_treatment_pred'])

In [ ]:
dur_df_t0 = dur_df(test_t0,0)

dur_df_t1 = dur_df(test_t1,1)
dur_df_t2 = dur_df(test_t2,2)
dur_df_t3 = dur_df(test_t3,3)

In [ ]:
#Добавляем к df временной интервал
dur_df_t0['t'] = 't0'
dur_df_t1['t'] = 't1' 
dur_df_t2['t'] = 't2'
dur_df_t3['t'] = 't3'

In [ ]:
#Соединяем получвшиеся прогнозы в один df
sum_df_dur = pd.concat([pd.concat([dur_df_t0,dur_df_t1]),pd.concat([dur_df_t2,dur_df_t3])])

In [ ]:
#Проверяем совпадает ли длинна
dur_df_t0.shape[0] + dur_df_t1.shape[0] + dur_df_t2.shape[0] + dur_df_t3.shape[0]

In [ ]:
sum_df_dur

In [ ]:
#На основе df, рисуем Violin_plots по Duration_treatment

import plotly.graph_objects as go

import pandas as pd

df = sum_df_dur.copy()

fig = go.Figure()

fig.add_trace(go.Violin(x = df['t'],
                        y=df['duration_treatment_true'],
                        legendgroup='True_values', scalegroup='Yes', name='Real duration',
                        side='negative',
                        line_color='blue')
             )
fig.add_trace(go.Violin(x = df['t'],
                        y=df['duration_treatment_pred'],
                        legendgroup='Predict_values', scalegroup='Yes', name='Predict duration',
                        line_color='orange')
             )
fig.update_traces(meanline_visible=True)
fig.update_layout(violingap=0, violinmode='overlay',autosize=True)
fig.show('notebook')

In [ ]:
fig.write_html('plotly_duration_last.html')